### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size

layer: Conv1d
func: <function parse_conv1d_layer at 0x7fd452c97f80>
layer: Conv2d
func: <function parse_conv2d_layer at 0x7fd452cb9050>
layer: Linear
func: <function parse_linear_layer at 0x7fd452cb9320>
layer: Softmax
func: <function parse_activation_layer at 0x7fd452cb93b0>
layer: ReLU
func: <function parse_activation_layer at 0x7fd452cb93b0>
layer: BatchNorm2d
func: <function parse_batchnorm_layer at 0x7fd452cb9440>
layer: BatchNorm1d
func: <function parse_batchnorm_layer at 0x7fd452cb9440>
handler args: ('NodeBlock',)
handler args: ('EdgeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)
layer: NodeBlock
func: <function parse_NodeBlock at 0x7fd452c9a710>
layer: EdgeBlock
func: <function parse_EdgeBlock at 0x7fd452c9a8c0>
layer: EdgeAggregate
func: <function parse_EdgeAggregate at 0x7fd452c9aa70>
layer: ResidualBlock
func: <function parse_ResidualBlock at 0x7fd452c9ac20>
layer: NodeEncoder
func: <f

### PyTorch Model

In [2]:
torch_model = InteractionNetwork(aggr="add", flow="source_to_target", hidden_size=40)
# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)

for name, submodule in torch_model.named_modules():
    if name != "":
        print(f"{name}: {submodule}")

O: ObjectModel(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=5, bias=True)
  )
)
O.layers: Sequential(
  (0): Linear(in_features=5, out_features=40, bias=True)
  (1): ReLU()
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): ReLU()
  (4): Linear(in_features=40, out_features=5, bias=True)
)
O.layers.0: Linear(in_features=5, out_features=40, bias=True)
O.layers.1: ReLU()
O.layers.2: Linear(in_features=40, out_features=40, bias=True)
O.layers.3: ReLU()
O.layers.4: Linear(in_features=40, out_features=5, bias=True)
R2: RelationalModel(
  (layers): Sequential(
    (0): Linear(in_features=15, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=1, bias=True)
  )
)
R2.layers: Sequential(
  (0): Lin

### HLS Model

In [3]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
# forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
# forward_dict["res_block"] = "ResidualBlock"
# forward_dict["R2"] = "EdgeBlock"

In [4]:
common_dim = 5
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
#         "node_dim": 3,
        "node_attr": 3,
        "node_dim": common_dim,
#         "edge_dim": 4,
        "edge_attr": 4,
    "edge_dim":common_dim
}

In [5]:
# output_dir = "test_GNN"
# config = config_from_pyg_model(torch_model,
#                                    default_precision="ap_fixed<32,12>",
#                                    default_index_precision='ap_uint<16>', 
#                                    default_reuse_factor=1)
# hls_model = convert_from_pyg_model(torch_model,
#                                        n_edge=graph_dims['n_edge'],
#                                        n_node=graph_dims['n_node'],
#                                        edge_attr=graph_dims['edge_attr'],
#                                        node_attr=graph_dims['node_attr'],
#                                        edge_dim=graph_dims['edge_dim'],
#                                        node_dim=graph_dims['node_dim'],
#                                        forward_dictionary=forward_dict, 
#                                        activate_final='sigmoid',
#                                        output_dir=output_dir,
#                                        hls_config=config)

In [6]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<16,8>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid',#'sigmoid',
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<16,8>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
PygModelReader node_dim: 5
PygModelReader edge_dim: 5
PygModelReader node_attr: 3
PygModelReader edge_attr: 4
node encoder layer_dict['n_in']: 3
node encoder layer_dict['n_out']: 5
edge encoder b4 update_dict["last_edge_update"]: edge_attr
edge encoder after update_dict["last_edge_update"]: layer5_out
type(update_dict): <class 'dict'>
nodeblock layer_dict: {'name': 'O', 'n_node': 28, 'n_edge': 37, 'node_dim': 5, 'edge_dim': 5, 'activate_final': 'false', 'activation': 'linear', 'n_layers': 3, 'out_dim': 5, 'class_name': 'NodeBlock', 'inputs': ['layer4_out', 'layer6_out'], 'outputs': ['layer7_out']}
NodeEncoder name: node_encoder
self.state_dict.keys(): odict_keys(['O.layers.0.weight', 'O.layers.0.bias', 'O.layers.2.weight', 'O.layers.2.bias', 'O.layers.4.weight', 'O.layers.4.bias', 'R2.layers.0.weight', 'R2.layers.0.bias', 'R2.layers.2.weight', 'R2.layers.2.bias',

In [7]:
hls_model.compile()

Writing HLS project
layer: <hls4ml.model.hls_layers.Input object at 0x7fd41f4529d0>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fd41f452b10>])
layer name: node_attr. func: None
layer: <hls4ml.model.hls_layers.Input object at 0x7fd41f4b2b50>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fd41f4b27d0>])
layer name: edge_attr. func: None
layer: <hls4ml.model.hls_layers.Input object at 0x7fd41f4b2c90>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fd41f4a4390>])
layer name: edge_index. func: None
layer: <hls4ml.model.hls_layers.NodeEncoder object at 0x7fd41f4a4850>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fd41f4c8f90>])
def_cpp: layer4_t layer4_out[N_LAYER_1_4*N_LAYER_2_4]
layer name: node_encoder. func: ['nnet::encoder<input_t, layer4_t, config4>(node_attr, layer4_out, w4, b4);']
layer: <hls4ml.model.hls_layers.Edg

Done
lib_name: firmware/myproject-f5b8AB86.so


# Evaluation and prediction: hls_model.predict(input)

### Data

In [8]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_dims = {
#         "n_node": 28,
#         "n_edge": 37,
#         "node_dim": 3,
#         "edge_dim": 4
#     }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [9]:
data = graphs[0]
print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")

print((np.abs(torch_pred- hls_pred)))
print(torch_pred)
print(hls_pred)

graphs len: 10
<class 'list'>
data.x shape:torch.Size([28, 3])
Node Encoder output mean: -0.12235080450773239. std: 0.45835477113723755
Edge Encoder output mean: 0.07100589573383331. std:0.2518100142478943
node residual: tensor([[-0.5178, -0.3714,  0.5937, -0.1547, -0.1660],
        [-0.0051, -0.5721,  0.8640, -0.7475,  0.6507],
        [-0.0099, -0.6286,  0.9308, -0.6840,  0.6740],
        [-0.0151, -0.6951,  1.0073, -0.6087,  0.7023],
        [-0.0157, -0.7598,  1.0865, -0.5522,  0.7281],
        [-0.0283, -0.8561,  1.1906, -0.4176,  0.7745],
        [-0.5294, -0.7916,  1.1116,  0.2237, -0.0050],
        [-0.4906, -0.2489,  0.1222, -0.4537, -0.3361],
        [ 0.0079, -0.3988,  0.4137, -1.1692,  0.3829],
        [ 0.0032, -0.4005,  0.4437, -1.1560,  0.3632],
        [-0.0051, -0.4033,  0.4851, -1.1423,  0.3337],
        [-0.5172, -0.3753,  0.5897, -0.1564, -0.1668],
        [-0.0120, -0.5740,  0.8665, -0.7408,  0.6444],
        [-0.0167, -0.6305,  0.9330, -0.6772,  0.6680],
        [

In [ ]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [ ]:
MSE_l = []
for data in graphs:
#     print(f"graphs len: {len(graphs)}")
#     print(type(data.hls_data))
#     print(f"data.x shape:{data.x.shape}")
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
#     print(f"torch pred shape: {torch_pred.shape}")
    hls_pred = hls_model.predict(data.hls_data)
#     print(f"hls_pred.shape: {hls_pred.shape}")
    MSE = mean_squared_error(torch_pred, hls_pred)
#     get_meanNstd(data, torch_pred, hls_pred)
    print(f"MSE: {MSE}")
    MSE_l.append(MSE)
#     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
#     print(f"MAPE: {MAPE}")

MSE_l = np.array(MSE_l)
print(f"MSE_l: {MSE_l}")
print(f"overall MSE: {np.mean(MSE_l)}")
print(MSE_l.shape)

In [ ]:
for data_instance in data.hls_data:
    print(data_instance.shape)
    
print(data.edge_index.shape)

In [ ]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict, n_graphs =1000):
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        if i >= n_graphs:
            break
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array
        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [ ]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

In [ ]:
from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
node_scaler = StandardScaler()
edge_scaler = StandardScaler()
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
#     print(type(graphs))

    for data in graphs: # fit data to scaler b4 feedforwarding
#         print(f"data type: {type(data)}")
        node_scaler.fit(data.x) 
        edge_scaler.fit(data.edge_attr) 
    
    for data in graphs:
       
        data.x = torch.from_numpy(node_scaler.transform(data.x)).float()
        data.edge_attr = torch.from_numpy(edge_scaler.transform(data.edge_attr)).float()
#         data.hls_data = [data.x, edge_attr, edge_index]
        data.hls_data[0] = node_scaler.transform(data.hls_data[0])
        data.hls_data[1] = edge_scaler.transform(data.hls_data[1])
        
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
        print(f"torch_pred shape: {torch_pred.shape}")
        hls_pred = hls_model.predict(data.hls_data)
        print(f"hls_pred shape: {hls_pred.shape}")
        MSE = mean_squared_error(torch_pred, hls_pred)
        print(f"MSE: {MSE}")
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
MSE_l = np.array(MSE_l)
print(f"MSE_l :{MSE_l}")
print(f"MSE means: {np.mean(MSE_l)}")

In [ ]:
MSE_l.shape

In [ ]:
torch_model.state_dict()